In [ ]:
import astropy.io.fits as fits
import numpy as np
import fitsio, os, sys
from astropy.table import Column
from astropy.table import Table
from math import cos,pi,log10
import matplotlib.pylab as plt
import healpy as hp
import pylab as py
from desitarget.geomask import radec_match_to 
#m1, m2, dist = radec_match_to(mtl, sdss[ind], return_sep=True) 


In [ ]:
path='/uufs/chpc.utah.edu/common/home/astro/dawson/sarahE/eboss/Dec2019/'
plotpath='/uufs/chpc.utah.edu/common/home/astro/dawson/sarahE/eboss/Dec2019/plots/'
#print(os.system('ls '+path+'*.fits'))

 eBOSS_QSO_hod_utah_eboss_SGC_v7_trimm_legacy_no-obs.dat.fits    ---> do not use any weight here
 eBOSS_QSO_hod_utah_eboss_SGC_v7_trimm_legacy_no-obs.ran.fits
 
 eBOSS_QSO_hod_utah_eboss_NGC_v7_trimm_legacy_no-obs.dat.fits
 eBOSS_QSO_hod_utah_eboss_NGC_v7_trimm_legacy_no-obs.ran.fits

 eBOSS_LRG_hod_utah_eboss_SGC_v7_trimm_legacy_no-obs.dat.fits
 eBOSS_LRG_hod_utah_eboss_SGC_v7_trimm_legacy_no-obs.ran.fits

 eBOSS_LRG_hod_utah_eboss_NGC_v7_trimm_legacy_no-obs.dat.fits
 eBOSS_LRG_hod_utah_eboss_NGC_v7_trimm_legacy_no-obs.ran.fits

 
 
 eBOSS_LRG_clustering_NGC_v7_COMP_BOSS.ran.fits ---> 
 eBOSS_LRG_clustering_SGC_v7_COMP_BOSS.ran.fits
 
 eBOSS_QSO_clustering_NGC_v7_COMP_BOSS.ran.fits ---> cols: 
 eBOSS_QSO_clustering_SGC_v7_COMP_BOSS.ran.fits
 
 eBOSS_QSO_clustering_NGC_v7_1_COMP_BOSS.ran.fits
 eBOSS_QSO_clustering_SGC_v7_1_COMP_BOSS.ran.fits
 
 eBOSS_QSO_clustering_SGC_v7_1_ids.dat.fits
 eBOSS_QSO_clustering_NGC_v7_1_ids.dat.fits
 
 weights_tiled_QSO_eBOSS_NGC_31x60_trimm_promoted_v7_no-obs.dat
 weights_tiled_QSO_eBOSS_SGC_31x60_trimm_promoted_v7_no-obs.dat
 
  These are ASCII files with columns EBOSS_TARGET_ID, RA, DEC, Z (neglect Z) and a sequence of 60 columns that contain the bitwise weights resulting from the 1860 fiber assignment runs. The first bit in the first of these 60 columns is the outcome of the actual eBOSS tiling run while the remaining bits are random realizations of the tiling. 
 
 
 # w_qso = WEIGHT_FKP*WEIGHT_SYSTOT*(WEIGHT_NOZ + WEIGHT_CP − 1)

In [ ]:
tgt='QSO'

cats=['eBOSS_'+tgt+'_hod_utah_eboss_SGC_v7_trimm_legacy_no-obs.dat.fits',
          'eBOSS_'+tgt+'_hod_utah_eboss_NGC_v7_trimm_legacy_no-obs.dat.fits',
          'eBOSS_'+tgt+'_clustering_SGC_v7_1_ids.dat.fits',
          'eBOSS_'+tgt+'_clustering_NGC_v7_1_ids.dat.fits']

#tgts=['QSO','LRG']


for name in cats:
    a = fits.open(path+name)
    cols = a[1].columns
    print(name,np.shape(a[1].data),cols.names)
    print('----------------------')
    

In [ ]:
fig = py.figure(figsize=(10, 5), dpi=100)
ax= fig.add_subplot(111)
tgt='QSO'
sq=fitsio.read(path+'eBOSS_'+tgt+'_hod_utah_eboss_SGC_v7_trimm_legacy_no-obs.dat.fits')
nq=fitsio.read(path+'eBOSS_'+tgt+'_hod_utah_eboss_NGC_v7_trimm_legacy_no-obs.dat.fits')

nnq=fitsio.read(path+'eBOSS_'+tgt+'_hod_utah_eboss_NGC_v7_trimm_legacy_no-obs.dat.fits')

ofs=2
xlim = (np.min(sq['RA'])-ofs,np.max(sq['RA'])+ofs )
ylim = (0,65)#(np.min(sq['DEC'])-ofs,np.max(sq['DEC'])+ofs )

ii = (sq['RA']+90 > 360)
py.plot(sq['RA'][~ii]+90,sq['DEC'][~ii],marker='.',markersize=0.1,linestyle='')
py.plot(nq['RA']+90,nq['DEC'],marker='.',markersize=0.1,linestyle='',color='orange')
py.plot(sq['RA'][ii]+90-360,sq['DEC'][ii],marker='.',markersize=0.1,linestyle='',color='blue')

ax.set_xlim(xlim)
ax.set_ylim(ylim)

#ax.set_title(plot_title)
ax.set_xlabel('R.A. [deg.]',fontsize=15)
ax.set_ylabel('Dec. [deg.]',fontsize=15)
ax.minorticks_on()
ax.tick_params(which='major', length=8, width=1.5, direction='in') 
ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')        

x_ax = ax.twiny()
x_ax.minorticks_on()
x_ax.tick_params(which='major', length=8, width=1.0, direction='in')
x_ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')     
x_ax.tick_params()
y_ax= plt.twinx()
y_ax.minorticks_on()
x_ax.set_xlim(xlim)
y_ax.set_ylim(ylim)
y_ax.tick_params(which='major', length=8, width=1.0, direction='in')
y_ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')     
#pylab.savefig(path+output_filename)
ax.grid()
plt.show()


In [ ]:
fig = py.figure(figsize=(10, 5), dpi=100)
ax= fig.add_subplot(111)
tgt='QSO'
sq=fitsio.read(path+'eBOSS_'+tgt+'_hod_utah_eboss_SGC_v7_trimm_legacy_no-obs.dat.fits')
nq=fitsio.read(path+'eBOSS_'+tgt+'_hod_utah_eboss_NGC_v7_trimm_legacy_no-obs.dat.fits')
nnq= fitsio.read(path+'eBOSS_QSO_clustering_NGC_v7_1_ids.dat.fits')

print('Orange:hod_utah_eboss_SGC_v7_trimm_legacy_no-obs.dat',len(nq))
print('Green: eBOSS_QSO_clustering_NGC_v7_1_ids.dat',len(nnq))
ofs=2
xlim = (100,270)#(np.min(sq['RA'])-ofs,np.max(sq['RA'])+ofs )
ylim = (10,65)#(np.min(sq['DEC'])-ofs,np.max(sq['DEC'])+ofs )

py.plot(sq['RA'],sq['DEC'],marker='.',markersize=0.1,linestyle='')
py.plot(nq['RA'],nq['DEC'],marker='.',markersize=0.1,linestyle='',color='orange')
py.plot(nnq['RA'],nnq['DEC'],marker='.',markersize=0.1,linestyle='',color='green')

ax.set_xlim(xlim)
ax.set_ylim(ylim)

#ax.set_title(plot_title)
ax.set_xlabel('R.A. [deg.]',fontsize=15)
ax.set_ylabel('Dec. [deg.]',fontsize=15)
ax.minorticks_on()
ax.tick_params(which='major', length=8, width=1.5, direction='in') 
ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')        

x_ax = ax.twiny()
x_ax.minorticks_on()
x_ax.tick_params(which='major', length=8, width=1.0, direction='in')
x_ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')     
x_ax.tick_params()
y_ax= plt.twinx()
y_ax.minorticks_on()
x_ax.set_xlim(xlim)
y_ax.set_ylim(ylim)
y_ax.tick_params(which='major', length=8, width=1.0, direction='in')
y_ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')     
#pylab.savefig(path+output_filename)
ax.grid()
plt.show()


In [ ]:
qso=fitsio.read(path+'eBOSS_QSO_clustering_NGC_v7_1_ids.dat.fits')
ran=fitsio.read(path+'eBOSS_QSO_clustering_NGC_v7_1_COMP_BOSS.ran.fits')

fig = py.figure(figsize=(10, 5), dpi=100)
ax= fig.add_subplot(111)

print('min/max z of QSOs:',np.min(qso['Z']),np.max(qso['Z']), 'Mean_Z=',np.mean(qso['Z']))
print('min/max z of RANs:',np.min(ran['Z']),np.max(ran['Z']), 'Mean_Z=',np.mean(ran['Z']))

#This gives a scale of 5.893 kpc/": from Ned Wright cosmo calc;
# conform in astropy

from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=100, Om0=0.307, Tcmb0=2.725)

scale = cosmo.kpc_proper_per_arcmin(np.mean(qso['Z'])).value/60

print('Angular scale @ average redshift of the sample: {} kpc/h/arcsec '.format(np.round(scale,2)))

print( '{} degree is 100 Mpc/h'.format(np.round(100000/scale/3600.,2)))

# Comoving distance at the average red
dc = cosmo.comoving_distance(np.mean(qso['Z']))

print('Comoving distance @ average redshift: {}, {}/h'.format(dc.value,dc.unit))

binsize=0.1
hq,qed = np.histogram(qso['Z'],bins=np.arange(np.min(qso['Z']),np.max(qso['Z']+binsize),binsize), density=True) 
hr,red = np.histogram(ran['Z'],bins=np.arange(np.min(ran['Z']),np.max(ran['Z']+binsize),binsize), density=True) 
#print(qed)
midz = (qed+np.arange(np.min(qso['Z']),np.max(qso['Z'])+binsize,binsize))/2
#print(midz)

####sanity check: 

#print(sum(hq/sum(hq)))   # should be 
#print(sum(hr/sum(hr)))
#print(sum(qso['NZ']*len(qso)))

#sys.exit()
plt.plot(midz[0:len(midz)-1],hq/sum(hq),color='orange')
plt.plot(midz[0:len(midz)-1],hr/sum(hr),color='blue',linestyle='-.')
xlim = (0.7,np.max(qso['Z']) )
ylim = (0.03,np.max(hr/sum(hr))+0.01 )

ax.set_xlim(xlim)
ax.set_ylim(ylim)

#ax.set_title(plot_title)
ax.set_xlabel('Z',fontsize=15)
ax.set_ylabel('N(z)',fontsize=15)
ax.minorticks_on()
ax.tick_params(which='major', length=8, width=1.5, direction='in') 
ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')        

x_ax = ax.twiny()
x_ax.minorticks_on()
x_ax.tick_params(which='major', length=8, width=1.0, direction='in')
x_ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')     
x_ax.tick_params()
y_ax= plt.twinx()
y_ax.minorticks_on()
x_ax.set_xlim(xlim)
y_ax.set_ylim(ylim)
y_ax.tick_params(which='major', length=8, width=1.0, direction='in')
y_ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')     
#pylab.savefig(path+output_filename)
ax.legend(['QSOs','Randoms'])
ax.grid()
plt.show()
  
    


In [ ]:
### get the L_bol and absolute i-magnitude of the quasars sample

# match the RA/DEC or target IDs with main eboss catalog 

# DD(s)  , s= sqrt(rp^2 + pi^2)

In [ ]:
#### post process of output anular pair count 
### binning in comoving distance
powmax=2.5
powmin=-1.5
delpow=0.2
edges=10.**(np.arange(powmin,powmax,delpow))
mids=[]
for i in range(len(edges)):
        mids.append((10**((powmin)+((i+1)*(delpow)))+10**((powmin)+(i*(delpow))))/2)



import glob
qso = fitsio.read(path+'eBOSS_QSO_clustering_NGC_v7_1_withS.fits')
DDpath = path+'/DD_QSO_v7_1_NGC/'
files = glob.glob(DDpath+'*_DD.fits')

from astropy.cosmology import FlatLambdaCDM
from math import sqrt, pi
cosmo = FlatLambdaCDM(H0=100, Om0=0.307, Tcmb0=2.725)

chunksize=10000
DD = np.zeros(len(edges)-1)
for i,f in enumerate(files):
     
     a = fitsio.read(DDpath+'spherematch.'+str(i)+'_DD.fits') # index1, index2, dist(in degrees)
     print('working on batch # {}, file:{}'.format(i,'spherematch.'+str(i)+'_DD.fits'))
     d = a['dist']
     ind2 = a['index2'] 
     z2 = qso['Z'][ind2]
     ind1 = a['index1']
     print(np.min(ind1),np.max(ind1),)
     s2 = qso['s'][ind2]
     s1 = qso['s'][ind1]
     pi = abs(s1-s2)
     rp = (s1+s2)*d*pi/180./2.
     ss  = np.sqrt(pi**2.+rp**2.)
     hist_s = np.histogram(ss, bins = edges)
     DD += hist_s[0]
     print(len(hist_s[0]))


In [216]:
print(DD,len(DD),len(edges),len(mids[0:19]))

t=Table()
col = Column(edges[0:19],name='low_edge',dtype='f4')
t.add_column(col,index=0)
col = Column(edges[1:20],name='up_edge',dtype='f4')
t.add_column(col,index=0)
col = Column(mids[0:19],name='Mid_s_mpch',dtype='f4')
t.add_column(col,index=0)
col = Column(DD,name='DD_s',dtype='i8')
t.add_column(col,index=0)
t.write(path+'DD_s_eBOSS_QSO_NGC_v7_1.fits')


[5.580000e+02 8.800000e+02 1.430000e+03 2.308000e+03 3.926000e+03
 5.624000e+03 9.058000e+03 1.367200e+04 2.101200e+04 3.209400e+04
 4.941600e+04 7.495000e+04 1.147380e+05 1.800880e+05 2.865060e+05
 4.737600e+05 7.489000e+05 1.177802e+06 1.868116e+06] 19 20 19


# DD(rp) ; rp=(s1+s2)*d*pi/180./2.

In [ ]:
#### post process of output anular pair count 
### binning in comoving distance
powmax=2.5
powmin=-1.5
delpow=0.2
edges=10.**(np.arange(powmin,powmax,delpow))
mids=[]
for i in range(len(edges)):
        mids.append((10**((powmin)+((i+1)*(delpow)))+10**((powmin)+(i*(delpow))))/2)



import glob
qso = fitsio.read(path+'eBOSS_QSO_clustering_NGC_v7_1_withS.fits')
DDpath = path+'/DD_QSO_v7_1_NGC/'
files = glob.glob(DDpath+'*_DD.fits')

from astropy.cosmology import FlatLambdaCDM
from math import sqrt, pi
cosmo = FlatLambdaCDM(H0=100, Om0=0.307, Tcmb0=2.725)

chunksize=10000
DD = np.zeros(len(edges)-1)
for i,f in enumerate(files):
     
     a = fitsio.read(DDpath+'spherematch.'+str(i)+'_DD.fits') # index1, index2, dist(in degrees)
     print('working on batch # {}, file:{}'.format(i,'spherematch.'+str(i)+'_DD.fits'))
     d = a['dist']
     ind2 = a['index2'] 
     z2 = qso['Z'][ind2]
     ind1 = a['index1']
     print(np.min(ind1),np.max(ind1),)
     s2 = qso['s'][ind2]
     s1 = qso['s'][ind1]
     pi = abs(s1-s2)
     rp = (s1+s2)*d*pi/180./2.
     ss  = np.sqrt(pi**2.+rp**2.)
     hist_s = np.histogram(ss, bins = edges)
     DD += hist_s[0]
     print(len(hist_s[0]))


In [ ]:
#w_qso = WEIGHT_FKP*WEIGHT_SYSTOT*(WEIGHT_NOZ + WEIGHT_CP − 1)
w_qso = qso['WEIGHT_FKP']*qso['WEIGHT_SYSTOT']*(qso['WEIGHT_NOZ'] + qso['WEIGHT_CP'] - 1)
print(np.min(w_qso),max(w_qso))

In [ ]:
#with open(path+'weights_tiled_QSO_eBOSS_NGC_31x60_trimm_promoted_v7_no-obs.dat','r'):
from astropy.io import ascii

data = ascii.read(path+'weights_tiled_QSO_eBOSS_NGC_31x60_trimm_promoted_v7_no-obs.dat')



In [ ]:
cols=data.columns
print(len(cols[0]),len(cols),len(data[0]))


In [ ]:
fig = py.figure(figsize=(10, 5), dpi=100)
ax= fig.add_subplot(111)
i=0
print(cols[6])
for col in range(6,7):
    plt.plot(cols[col])
    i +=1
    print((np.min(cols[col]),np.max(cols[col]) ))
#ra=cols[1]
#dec=cols[2]
#plt.plot(ra,dec,marker='.',color='red',linestyle='',markersize=0.1)
xlim = (np.min(ra)-ofs,np.max(ra)+ofs )
ylim = (np.min(dec)-ofs,np.max(dec)+ofs )


ax.set_xlim(xlim)
ax.set_ylim(ylim)

#ax.set_title(plot_title)
ax.set_xlabel('times',fontsize=15)
ax.set_ylabel('',fontsize=15)
ax.minorticks_on()
ax.tick_params(which='major', length=8, width=1.5, direction='in') 
ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')        

x_ax = ax.twiny()
x_ax.minorticks_on()
x_ax.tick_params(which='major', length=8, width=1.0, direction='in')
x_ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')     
x_ax.tick_params()
y_ax= plt.twinx()
y_ax.minorticks_on()
x_ax.set_xlim(xlim)
y_ax.set_ylim(ylim)
y_ax.tick_params(which='major', length=8, width=1.0, direction='in')
y_ax.tick_params(which='minor', length=6, color='#000033', width=1.0, direction='in')     
#pylab.savefig(path+output_filename)
ax.grid()
plt.show()
  
    